##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
print(tf.config.list_physical_devices('CPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

from tensorflow.python.framework.ops import disable_eager_execution, enable_eager_execution
#disable_eager_execution()
#enable_eager_execution()

#from tensorflow.python.compiler.mlcompute import mlcompute
#mlcompute.set_mlc_device(device_name='cpu') # does not work,
# Same issue as https://developer.apple.com/forums/thread/684263

# eagerly: ref: https://www.tensorflow.org/guide/intro_to_graphs
tf.config.run_functions_eagerly(False) # Only seems to have effect when restarting the jupyter notebook.
#tf.compat.v1.disable_eager_execution()
print(f'tf.executing_eagerly() = {tf.executing_eagerly()}')  # Gives True when line above set it to False!
# I get on a Apple MacBook Pro 16" M1 Max, for the model fitting (training) the process times:
# When executing_eagerly = True: 109s. 
# When executing_eagerly = False: 40s.

# So eagerly slows it down by a factor 2.7!
# It's wrong that 'eagerly' is with GPU, 
# This page seems to think eagerly ~ GPU use. Same slowdown noted for 'eagerly' mode.
# https://github.com/apple/tensorflow_macos/issues/12
print(f"tf.device('/cpu:0') = {tf.device('/cpu:0')}")
print(f"tf.device('/gpu:0') = {tf.device('/gpu:0')}")

# https://stackoverflow.com/questions/72988252/gpu-much-slower-than-cpu-for-text-processing-using-tensorflow-macos
# mentions slower performance with GPU than with CPU when using RNNs and faster GPU than CPU perf when using CNNs.
# So try to find an example with CNNs

if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

TensorFlow version: 2.9.2
Num CPUs Available:  1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
tf.executing_eagerly() = True
tf.device('/cpu:0') = <tensorflow.python.eager.context._EagerDeviceContext object at 0x15db9ec00>
tf.device('/gpu:0') = <tensorflow.python.eager.context._EagerDeviceContext object at 0x15db9ed80>
TensorFlow **IS** using the GPU


2022-08-28 13:28:06.576171: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-28 13:28:06.576298: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.07257038, -0.10141516, -0.4927325 ,  0.21502802, -0.65959966,
         0.30831987, -0.42616212,  0.38444662, -0.06011833, -0.04235578]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.09703071, 0.09427186, 0.06374338, 0.12936346, 0.05394679,
        0.14201288, 0.06813123, 0.15324602, 0.09824649, 0.10000719]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

1.9518374

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [11]:
import datetime
#log_dir = "./" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = '/tmp/tflearn_logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [12]:
%%time
#tf.data.experimental.enable_debug_mode()  # <-- This was suggested by running the next line 
# after having done: tf.config.run_functions_eagerly(True) at the beginning of this notebook.
# But debug mode slows it down further to 3min 9s.
model.fit(x_train, y_train, epochs=2)

Epoch 1/2


2022-08-28 13:28:08.204823: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-28 13:28:08.361159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 10s 4ms/step - loss: 0.2871 - accuracy: 0.9162
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1326 - accuracy: 0.9609
CPU times: user 13.6 s, sys: 9.92 s, total: 23.5 s
Wall time: 17.4 s


In [13]:
%tensorboard --logdir /tmp/tflearn_logs


Reusing TensorBoard on port 6010 (pid 81357), started 15:15:45 ago. (Use '!kill 81357' to kill it.)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [14]:
%%time
model.evaluate(x_test,  y_test, verbose=2)

2022-08-28 13:28:25.567765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 - 1s - loss: 0.1061 - accuracy: 0.9686 - 1s/epoch - 4ms/step
CPU times: user 887 ms, sys: 659 ms, total: 1.55 s
Wall time: 1.14 s


[0.10607685148715973, 0.9686000347137451]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.43537534e-06, 8.34063201e-08, 1.44424164e-04, 9.96698625e-04,
        9.67058078e-09, 5.54517192e-06, 7.44067030e-11, 9.98783410e-01,
        7.65812638e-06, 5.97421968e-05],
       [1.20726418e-05, 1.03082578e-03, 9.97815132e-01, 8.29603465e-04,
        3.77470138e-10, 2.60393543e-04, 3.34908764e-05, 1.56882933e-08,
        1.83650282e-05, 1.45679968e-09],
       [8.80379448e-05, 9.84784722e-01, 2.42252415e-03, 5.78742183e-04,
        2.56666128e-04, 5.50678524e-04, 1.43326190e-03, 6.46806834e-03,
        3.22972820e-03, 1.87539583e-04],
       [9.99649882e-01, 1.75009893e-06, 4.50250900e-06, 1.46790558e-06,
        6.71439693e-06, 1.82070562e-05, 8.08926052e-05, 2.32651350e-04,
        5.62336169e-08, 3.91719414e-06],
       [1.19267082e-04, 8.88039153e-07, 1.40398392e-04, 1.39780923e-06,
        9.93064463e-01, 1.43474435e-05, 6.85499035e-05, 9.44985542e-04,
        3.32609925e-05, 5.61256940e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
